In [2]:
import pandas as pd

# Load data and set index
df = pd.read_csv('ohlc_1m_data_last_day.csv', parse_dates=['timestamp'])
df.set_index('timestamp', inplace=True)

# Stochastic Oscillator calculation
n = 14  
df['Lowest_Low'] = df['low'].rolling(window=n).min()
df['Highest_High'] = df['high'].rolling(window=n).max()
df['%K'] = ((df['close'] - df['Lowest_Low']) / (df['Highest_High'] - df['Lowest_Low'])) * 100
df['%D'] = df['%K'].rolling(window=3).mean()

# Shifted values for crossover detection
df['%K_shift'] = df['%K'].shift(1)
df['%D_shift'] = df['%D'].shift(1)

# Signal generation
df['Signal'] = 0  
df['Signal'] = df.apply(
    lambda row: 1 if (row['%K'] > row['%D']) and (row['%K_shift'] <= row['%D_shift']) else (
        -1 if (row['%K'] < row['%D']) and (row['%K_shift'] >= row['%D_shift']) else 0
    ), axis=1
)

# Remove "Hold" signals (i.e., where Signal == 0)
df = df[df['Signal'] != 0]

# Explicitly label signals as 'Buy' or 'Sell'
df['SC_Signals'] = df['Signal'].apply(lambda x: 'Buy' if x == 1 else 'Sell')

# Resample to 4-hour intervals
df_4h = df.resample('4H').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum',
    '%K': 'last',
    '%D': 'last',
    'Signal': 'last',  
    'SC_Signals': 'last' 
}).dropna()

# Save to CSV
df_4h.to_csv('stochastic_4h_signals.csv')


C:\Users\MTariq\AppData\Local\Temp\ipykernel_8744\3031975852.py:33: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_4h = df.resample('4H').agg({
